# BETO  EXITO.COM
<center><img src="https://www.grupoexito.com.co/sites/default/files/2019-12/logo-exito-com.png"></center>
<center><img src="https://1.bp.blogspot.com/-Oc6Da1nmTkE/X1EtuFg31oI/AAAAAAAAdFE/0TPEOjf9Wico8EGK-FyGBdOHb9meeyD9gCNcBGAsYHQ/s829/118823936_1170125176699612_3119017095284167246_n.jpg"></center>

In [ ]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
# For reproducibility
np.random.seed(1237)

In [ ]:
Detractores = pd.read_excel('Concatenados.xlsx')
Detractores.head()

,IdChat,Chat
0,0003UaFUQUYKUCX1,Mi pedido excedió el tiempo de entrega
1,0003UaFUQUYKUD3R,Bueno Buenos días es que me gustaría solicitar...
2,0003UaFUQUYKUD4W,se puede solicitar la tarjeta éxito en línea s...
3,0003UaFUQUYKUD8X,Mi pedido excedió el tiempo de entrega
4,0003UaFUQUYKUD9Y,"Buenos días, el domingo entregaron mi compra p..."


In [ ]:
Detractores.isna().sum()

IdChat    0
Chat      0
dtype: int64

In [ ]:
Tipologia = pd.read_excel('TipologiaProc.xlsx')
Tipologia.head()

,IdChat,ProSer,Proceso,Tecnologia
0,0003UaFUQUYKUD9Y,NaN,Inconformidad con procesos de entrega,NaN
1,0003UaFUQUYKUCX1,NaN,Pedido Sin Entregar: Fuera del tiempo establecido,NaN
2,0003UaFUQUYKUGPD,NaN,Caso sin seguimiento: Dentro de tiempo estable...,NaN
3,0003UaFUQUYKUJXM,NaN,Pedido Sin Entregar: Fuera del tiempo establecido,NaN
4,0003UaFUQUYKUU8P,NaN,Transportadora: Falta de comunicación con el c...,NaN


In [ ]:
Tipologia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IdChat      279 non-null    object 
 1   ProSer      0 non-null      float64
 2   Proceso     279 non-null    object 
 3   Tecnologia  0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 8.8+ KB


In [ ]:
Detractores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16814 entries, 0 to 16813
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   IdChat  16814 non-null  object
 1   Chat    16814 non-null  object
dtypes: object(2)
memory usage: 262.8+ KB


In [ ]:
#Union de las datas
Datos = Tipologia.merge (Detractores, on='IdChat')
Datos.head(20)

,IdChat,ProSer,Proceso,Tecnologia,Chat
0,0003UaFUQUYKUD9Y,NaN,Inconformidad con procesos de entrega,NaN,"Buenos días, el domingo entregaron mi compra p..."
1,0003UaFUQUYKUCX1,NaN,Pedido Sin Entregar: Fuera del tiempo establecido,NaN,Mi pedido excedió el tiempo de entrega
2,0003UaFUQUYKUGPD,NaN,Caso sin seguimiento: Dentro de tiempo estable...,NaN,buenos dias por favor su ayuda solicito se rev...
3,0003UaFUQUYKUJXM,NaN,Pedido Sin Entregar: Fuera del tiempo establecido,NaN,ok es q tengo un pedido q deberia haber llegad...
4,0003UaFUQUYKUU8P,NaN,Transportadora: Falta de comunicación con el c...,NaN,buenos dias Queria preguntar por una compra qu...
5,0003UaFUQUYKV712,NaN,Funcionarios Exito,NaN,Hola buenos días quisiera saber si yo puedo pa...
6,0003UaFUQUYKVADH,NaN,Pedido Sin Entregar: Fuera del tiempo establecido,NaN,buen dia El 21 de nov hice un pedido por exito...
7,0003UaFUQUYKVAUH,NaN,Inconformidad con procesos de entrega,NaN,Tengo un pedido pendiente que realice el dia s...
8,0003UaFVD082XYUW,NaN,Pedido Sin Entregar: Dentro de los tiempo esta...,NaN,Mira lo que pasa es que el 29 de noviembre com...
9,0003UaFVD082XYX8,NaN,Caso sin seguimiento: Dentro de tiempo estable...,NaN,"Buenas Tardes Señora Milena, ?? le doy la bien..."


In [ ]:
Datos.Proceso.unique()

array(['Inconformidad con procesos de entrega',
       'Pedido Sin Entregar: Fuera del tiempo establecido',
       'Caso sin seguimiento: Dentro de tiempo establecido',
       'Transportadora: Falta de comunicación con el cliente',
       'Funcionarios Exito',
       'Pedido Sin Entregar: Dentro de los tiempo establecidos',
       'Reapertura de caso (Direccion no gestionada a la transportadora)',
       'Faltante', 'Compra y Recoge : No notifica', 'Cancelacion: Fraude',
       'Inconformidad por cancelacion de pedido', 'Cambios: No aplica ',
       'Exito: Fallo en estado de pedido', 'Envio de evidencias',
       'Envio de hoja laboral', 'Cambio inmediato de direccion erronea',
       'No llega producto', 'Solicita informacion de cobertura',
       'Inconformidad por envio de envidencia del caso',
       'Muy costoso el valor del envio',
       'Caso sin seguimiento: Fuera de los tiempos establecidos',
       'Caso con seguimiento: Cerrado', 'Cancelacion sin autorizacion',
       'Fal

<center><img src="https://media1.giphy.com/media/Wkr0EKIsWitJm/giphy.gif?cid=ecf05e47idc72md36psdrf7w8nzhxzd7l0tb2mnvwuw10ghy&rid=giphy.gif"></center>
<center> ¡Qué comience el juego!</center>

# Problemas con Proceso
<img src="https://www.lineadatascan.com/wp-content/uploads/2020/05/plantilla-arti%CC%81culo.jpg">

Ahora se divide la data en los tres problemas que aquejan a nuestros clientes
1. Proceso o Servicio
2. Proceso
3. Tecnología

In [ ]:
#Problemas con producto o servicio
colp = ['IdChat','Chat', 'Proceso']
df = Datos[colp]
df.head()

,IdChat,Chat,Proceso
0,0003UaFUQUYKUD9Y,"Buenos días, el domingo entregaron mi compra p...",Inconformidad con procesos de entrega
1,0003UaFUQUYKUCX1,Mi pedido excedió el tiempo de entrega,Pedido Sin Entregar: Fuera del tiempo establecido
2,0003UaFUQUYKUGPD,buenos dias por favor su ayuda solicito se rev...,Caso sin seguimiento: Dentro de tiempo estable...
3,0003UaFUQUYKUJXM,ok es q tengo un pedido q deberia haber llegad...,Pedido Sin Entregar: Fuera del tiempo establecido
4,0003UaFUQUYKUU8P,buenos dias Queria preguntar por una compra qu...,Transportadora: Falta de comunicación con el c...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279 entries, 0 to 278
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IdChat   279 non-null    object
 1   Chat     279 non-null    object
 2   Proceso  279 non-null    object
dtypes: object(3)
memory usage: 8.7+ KB


In [ ]:
#Valores nulos en la categoria
df.isna().sum()

IdChat     0
Chat       0
Proceso    0
dtype: int64

In [ ]:
# Remove a row if any of the three remaining columns are missing
df = df.dropna()
df.isna().sum()

IdChat    0
Chat      0
ProSer    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347 entries, 0 to 346
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   IdChat  347 non-null    object
 1   Chat    347 non-null    object
 2   ProSer  347 non-null    object
dtypes: object(3)
memory usage: 10.8+ KB


In [ ]:
possible_labels = df.Proceso.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Cambio de correo inmediato': 29,
 'Cambio inmediato de direccion erronea': 15,
 'Cambios: No aplica ': 11,
 'Cancelacion': 25,
 'Cancelacion sin autorizacion': 22,
 'Cancelacion: Fraude': 9,
 'Caso con seguimiento: Cerrado': 21,
 'Caso sin seguimiento: Dentro de tiempo establecido': 2,
 'Caso sin seguimiento: Fuera de los tiempos establecidos': 20,
 'Compra y Recoge : No notifica': 8,
 'Envio de evidencias': 13,
 'Envio de hoja laboral': 14,
 'Exito: Fallo en estado de pedido': 12,
 'Fallo al seleccionar el método de pago': 24,
 'Fallo en facturacion': 23,
 'Faltante': 7,
 'Funcionarios Exito': 4,
 'Inconformidad con procesos de compra': 28,
 'Inconformidad con procesos de entrega': 0,
 'Inconformidad por cancelacion de pedido': 10,
 'Inconformidad por envio de envidencia del caso': 18,
 'Muy costoso el valor del envio': 19,
 'No llega producto': 16,
 'Pedido Sin Entregar: Dentro de los tiempo establecidos': 5,
 'Pedido Sin Entregar: Fuera del tiempo establecido': 1,
 'Reapertura de 

In [ ]:
df['label'] = df.Proceso.replace(label_dict)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['label'].value_counts()

2     79
1     46
5     38
10    17
16    13
20    11
3     10
21     9
15     9
11     6
9      4
25     4
0      4
8      3
12     3
7      2
4      2
14     2
13     2
17     2
19     2
22     2
27     2
28     1
18     1
6      1
23     1
24     1
26     1
29     1
Name: label, dtype: int64

In [ ]:
df.columns

Index(['IdChat', 'Chat', 'ProSer', 'label'], dtype='object')

In [ ]:
co = ['Chat','label', 'Proceso']
df = df[co]
df.head()

,Chat,label,Proceso
0,"Buenos días, el domingo entregaron mi compra p...",0,Inconformidad con procesos de entrega
1,Mi pedido excedió el tiempo de entrega,1,Pedido Sin Entregar: Fuera del tiempo establecido
2,buenos dias por favor su ayuda solicito se rev...,2,Caso sin seguimiento: Dentro de tiempo estable...
3,ok es q tengo un pedido q deberia haber llegad...,1,Pedido Sin Entregar: Fuera del tiempo establecido
4,buenos dias Queria preguntar por una compra qu...,3,Transportadora: Falta de comunicación con el c...


In [ ]:
print(df.shape)

(279, 3)


In [ ]:
class_counts = df['label'].value_counts()
class_counts

2     79
1     46
5     38
10    17
16    13
20    11
3     10
21     9
15     9
11     6
9      4
25     4
0      4
8      3
12     3
7      2
4      2
14     2
13     2
17     2
19     2
22     2
27     2
28     1
18     1
6      1
23     1
24     1
26     1
29     1
Name: label, dtype: int64

In [ ]:
class_weights = len(class_counts)/class_counts
df = df.sample(
    n=class_counts.max()*len(class_counts),
    weights=df['label'].map(class_weights), 
    replace=True)

In [ ]:
df['label'].value_counts()

4     97
25    95
26    95
3     91
29    90
21    87
27    87
20    87
23    87
9     86
17    85
13    83
6     82
18    81
19    81
14    81
2     81
16    79
8     77
22    77
10    75
11    75
0     71
5     69
24    66
1     66
15    65
28    63
7     57
12    54
Name: label, dtype: int64

In [ ]:
df['Proceso'].value_counts()

Funcionarios Exito                                                  97
Cancelacion                                                         95
Seguimiento caso                                                    95
Transportadora: Falta de comunicación con el cliente                91
Cambio de correo inmediato                                          90
Caso sin seguimiento: Fuera de los tiempos establecidos             87
Caso con seguimiento: Cerrado                                       87
Fallo en facturacion                                                87
pedido incompleto                                                   87
Cancelacion: Fraude                                                 86
Solicita informacion de cobertura                                   85
Envio de evidencias                                                 83
Reapertura de caso (Direccion no gestionada a la transportadora)    82
Caso sin seguimiento: Dentro de tiempo establecido                  81
Inconf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', stop_words=stopwords.words("spanish"), ngram_range=(1, 5)) #parametrps para vectorizar y asignar pesos a las palabras

In [ ]:
features = tfidf.fit_transform(df.Chat).toarray()
labels = df.label
features.shape

(2370, 52707)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier

In [ ]:
datos = df.Chat

In [ ]:
tipificaciones=df.Proceso

In [ ]:
tipificaciones.value_counts()

Funcionarios Exito                                                  97
Cancelacion                                                         95
Seguimiento caso                                                    95
Transportadora: Falta de comunicación con el cliente                91
Cambio de correo inmediato                                          90
Caso sin seguimiento: Fuera de los tiempos establecidos             87
Caso con seguimiento: Cerrado                                       87
Fallo en facturacion                                                87
pedido incompleto                                                   87
Cancelacion: Fraude                                                 86
Solicita informacion de cobertura                                   85
Envio de evidencias                                                 83
Reapertura de caso (Direccion no gestionada a la transportadora)    82
Caso sin seguimiento: Dentro de tiempo establecido                  81
Inconf

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datos, tipificaciones)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
red=MLPClassifier(max_iter=40000, hidden_layer_sizes=(10))

In [ ]:
red.fit(X_train_tfidf, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=10, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=40000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
red.score(X_train_tfidf, y_train)

1.0

In [ ]:
interaccion = ("Señorita mi pedido no ha llegad y reviso en la pagina y me dice que mi pedido esta cancelado. Es el triado de mis hijos y no veo porque me cancelan ")
print(red.predict(count_vect.transform([interaccion])))

['Cancelacion']


In [ ]:
interaccion = ("Buenas tardes, quiero conocer estado de mi solicitud No. 00524715 Permitame un momento por favor, gracias y PQR 00536346 Hola..? Un momento más Le pido disculpas por la demora El caso que usted me dice esta cerrado por que el trámite se lleva con el caso No 00524715 Ya esta siendo gestionado, las evidencias estan en el centro de servicios y estamos a la espera de que ellos nos confirmen cómo vamos a proceder Ya le escribí al gestor que tiene el caso para que el valide con el centro de servicios que ha pasado y le envíe a su correo electrónico información Gracias. No obstante, llevo casi dos semanas es este cruce de chats sin ningún progreso. La semana pasada me dijeron que habían priorizado y que el proveedor me llamaría, ahora que me van a enviar un correo electrónico. Como entenderá, este manejo de cliente sin respuestas concretas más allá de tranquilizar enoja más. Los procesos de garantía son demorados Por favor evitenme llevar el tema a la SIC por un asunto tan básico. Tienen un tiempo de 30 días hábiles Por favor indíqueme en qué clausulado de los términos de garantía de Exito.com habla de los 30 días para atender las garantías.")
print(red.predict(count_vect.transform([interaccion])))

['Caso sin seguimiento: Dentro de tiempo establecido']


In [ ]:
interaccion = ("Buenas tardes, quiero conocer estado de mi solicitud No. 00524715 Permitame un momento por favor, gracias y PQR 00536346 Hola..? Un momento más Le pido disculpas por la demora El caso que usted me dice esta cerrado por que el trámite se lleva con el caso No 00524715 Ya esta siendo gestionado, las evidencias estan en el centro de servicios y estamos a la espera de que ellos nos confirmen cómo vamos a proceder Ya le escribí al gestor que tiene el caso para que el valide con el centro de servicios que ha pasado y le envíe a su correo electrónico información Gracias. No obstante, llevo casi dos semanas es este cruce de chats sin ningún progreso. La semana pasada me dijeron que habían priorizado y que el proveedor me llamaría, ahora que me van a enviar un correo electrónico. Como entenderá, este manejo de cliente sin respuestas concretas más allá de tranquilizar enoja más. Los procesos de garantía son demorados Por favor evitenme llevar el tema a la SIC por un asunto tan básico. Tienen un tiempo de 30 días hábiles Por favor indíqueme en qué clausulado de los términos de garantía de Exito.com habla de los 30 días para atender las garantías.")
print(red.predict(count_vect.transform([interaccion])))

['Fallo en el cambio en metodo de entrega']


In [ ]:
interaccion = ("Buenas tardes quiero preguntar lo de un pedido a domicilio que no ha llegado hace mucho tiempo con que número de cédula solicito? 1032357859 permitame valido También si me regalar el número de guía de la empresa prestadora de envío en este caso es coordinadora para así mismo yo también hacer seguimiento gracias me podría validar  la dirección de entrega ya que verificando la guía de coordinadora no la ha podido ubicar Carrera 13 #142 -32 CEDRITOS Bogotá  es en casa o unidad? Casa supermercado MAXXITIENDAS Antes estaba con dirección errada carrera 13 #42 -32 porque la dirección si le quedo errada debemos validar algunos datos para que le generen el cambio de dirección Ya el jueves lo hicieron ustedes Por lo tanto solo nesectio saber que ha pasado con el pedido y le dieron radicado para ese cambio?")
print(red.predict(count_vect.transform([interaccion])))

['Seguimiento caso']


In [ ]:
interaccion = ("Buenas tardes quiero preguntar lo de un pedido a domicilio que no ha llegado hace mucho tiempo con que número de cédula solicito? 1032357859 permitame valido También si me regalar el número de guía de la empresa prestadora de envío en este caso es coordinadora para así mismo yo también hacer seguimiento gracias me podría validar  la dirección de entrega ya que verificando la guía de coordinadora no la ha podido ubicar Carrera 13 #142 -32 CEDRITOS Bogotá  es en casa o unidad? Casa supermercado MAXXITIENDAS Antes estaba con dirección errada carrera 13 #42 -32 porque la dirección si le quedo errada debemos validar algunos datos para que le generen el cambio de dirección Ya el jueves lo hicieron ustedes Por lo tanto solo nesectio saber que ha pasado con el pedido y le dieron radicado para ese cambio?")
print(red.predict(count_vect.transform([interaccion])))

['Seguimiento caso']
